In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

import requests
from bs4 import BeautifulSoup
import re
import os
import time
import csv

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path2data = 'data1/'

# Make a data folder if does not already exist
import os
if not os.path.exists(path2data):
    os.makedirs(path2data)

# Determine IDs of articles to scrape
ELife since 2014

In [ ]:
# Load database of available articles
df = pd.read_csv('PMC-ids.csv')

In [ ]:
df_keep = df[df['Journal Title']=='eLife']
df_keep = df_keep[df_keep['Year']>=2018]
df_keep.reset_index(inplace=True, drop=True)

In [ ]:
df_keep.head()

# Scrape all

In [ ]:
apikey = open('apikey.txt', 'r').read()
db = 'pmc'
base = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?'

In [ ]:
# Load keywords
with open('keywords.csv', 'r') as f:
    reader = csv.reader(f)
    terms = list(reader)[0]

In [ ]:
# Number of characters to extract around keywords
span_buffer = 100

In [ ]:
# Saving parameters
N_previous = 0
N_chunk = 10000
N_save = 100

In [ ]:
# Load previous computation
if N_previous > 0:
    dfs_articles = {}
    for k in terms:
        csv_name = '{:s}{:s}_{:d}.csv'.format(path2data, k, N_previous)
        dfs_articles[k] = [pd.read_csv(csv_name, index_col=0)]
else:
    dfs_articles = defaultdict(list)

for i, row in df_keep.loc[N_previous+1:].iterrows():
    # Get full text of 1 paper
    pmcid = row['PMCID']
    s = '{:s}db={:s}&id={:s}'.format(base, db, pmcid, apikey)
    out = requests.get(s)
    bs = BeautifulSoup(out.content, 'lxml')

    # DFs of terms
    for term in terms:
        dict_term = defaultdict(list)
        for s in re.finditer(term, out.text, re.IGNORECASE):
            save_span = s.span()
            sent = out.text[(save_span[0] - span_buffer):(save_span[1] + span_buffer)]

            dict_term['PMCID'].append(pmcid)
            dict_term['sentence'].append(sent)
        dfs_articles[term].append(pd.DataFrame(dict_term))

    # Save output every N
    if (i % N_save == 0) & (i > 0):
        print(i)
        for k in dfs_articles.keys():
            df_save = pd.concat(dfs_articles[k])
            df_save.to_csv('{}{:s}_{:d}.csv'.format(path2data, k, i))

            # Delete last file Unless 
            if (i-N_save) % N_chunk > 0:
                os.remove('{}{:s}_{:d}.csv'.format(path2data, k, i - N_save))

        if i % N_chunk == 0:
            if i > 0:
                dfs_articles = defaultdict(list)
            
# Save output when finish
for k in dfs_articles.keys():
    df_save = pd.concat(dfs_articles[k])
    df_save.to_csv('{}{:s}_{:d}.csv'.format(path2data, k, i))
    os.remove('{}{:s}_{:d}.csv'.format(path2data, k, int(np.round(i-(N_save/2-1), -2))))